# Test Tools and Models

## Running this notebook

To run this in VS Code:
1. Open a terminal
2. Navigate to `./backend`
3. Configure poetry to set up a venv environment by running `poetry config virtualenvs.in-project true`
4. Remove any existing poetry venv by running `poetry remove --all`
5. Run `poetry install` to install the dependencies in the virtual environment
6. In Select Kernel dropdown in the top right corner of VS Code, select `Python Environments` and select the venv environment you created at `backend/.venv/bin/python`
7. Run the notebook by clicking on the play button in the top right corner of VS Code or running `jupyter notebook test_notebook.ipynb`

In [1]:
import sys
import os
import json
import logging

# Get the absolute path to the src directory
src_path = os.path.abspath('src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from dotenv import load_dotenv
load_dotenv('src/.env')
os.environ['MILVUS_HOST'] = 'localhost'
os.environ['OLLAMA_HOST'] = 'localhost'
from src.models import get_model, bind_tools, handle_tool_call, handle_streaming_tool_call
from src.tools import retrieve_context, multiply
from src.langgraph_agent import create_agent_graph, run_agent_graph, run_agent_graph_streaming
from src.app import ChatRequest, prompt_templates

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:src.tools:Initialized embedding model: all-MiniLM-L6-v2
INFO:src.tools:Connected to Milvus at localhost:19530
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:tools:Initialized embedding model: all-MiniLM-L6-v2
INFO:tools:Connected to Milvus at localhost:19530


In [6]:
# test_notebook.ipynb (Revised)
from langchain.schema import HumanMessage

async def test_agent_graph_streaming(user_prompt, agent_graph):
    initial_state = {
        "messages": [HumanMessage(content=user_prompt)],
    }

    # Run the agent and collect chunks
    collected = []
    async for chunk in run_agent_graph_streaming(agent_graph, initial_state):
        collected.append(chunk)
        print("Chunk:", chunk)  # Print each chunk as it arrives

    final_response = "".join(collected)
    return final_response

# Execute the test in Jupyter:
assistant_system_prompt = prompt_templates["assistant_system_prompt"]
tools = [multiply, retrieve_context]
agent_graph = create_agent_graph(tools, assistant_system_prompt, True) # Need to set Agent Graph to True to enable streaming
# Test the agent graph with a user prompt
user_prompt = "What is the phone number for DataNinja Support?"
await test_agent_graph_streaming(user_prompt, agent_graph)  

DEBUG:src.langgraph_agent:Running the Agent Graph Streaming Function...
INFO:src.langgraph_agent:Running assistant node
INFO:src.langgraph_agent:Current messages: ['human']
INFO:src.langgraph_agent:Adding system prompt to message history
INFO:root:Calling Ollama model with name: qwen2:7b
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x17ac87680>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:src.langgraph_agent:Running final answer async (streaming) node
INFO:root:Calling Ollama model with name: qwen2:7b
DEBUG:src.langgraph_agent:Debug Stream: <async_generator object final_answer_async.<locals>.wrapped_stream at 0x17bc96b20>
DEBUG:src.langgraph_agent:Running streaming wrapper...
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x17ac89ac0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b

Chunk:  The
Chunk:  phone
Chunk:  number
Chunk:  for
Chunk:  Data
Chunk: N
Chunk: inja
Chunk:  Support
Chunk:  is
Chunk:  
Chunk: 1
Chunk: -


DEBUG:src.langgraph_agent:Streamed chunk: content='8' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk: 8
DEBUG:src.langgraph_agent:Streamed chunk: content='0' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk: 0
DEBUG:src.langgraph_agent:Streamed chunk: content='0' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk: 0
DEBUG:src.langgraph_agent:Streamed chunk: content='-D' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk: -D
DEBUG:src.langgraph_agent:Streamed chunk: content='ATA' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk: ATA
DEBUG:src.langgraph_agent:Streamed chunk: content='-N' additional_kwargs={} res

Chunk: 8
Chunk: 0
Chunk: 0
Chunk: -D
Chunk: ATA
Chunk: -N
Chunk: IN
Chunk: JA
Chunk: ,
Chunk:  and
Chunk:  this


DEBUG:src.langgraph_agent:Streamed chunk: content=' support' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk:  support
DEBUG:src.langgraph_agent:Streamed chunk: content=' service' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk:  service
DEBUG:src.langgraph_agent:Streamed chunk: content=' operates' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk:  operates
DEBUG:src.langgraph_agent:Streamed chunk: content=' ' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk:  
DEBUG:src.langgraph_agent:Streamed chunk: content='2' additional_kwargs={} response_metadata={} id='run-67ab1ff0-f5d2-4dd7-b235-cb593f210b49'
DEBUG:src.langgraph_agent:Debug Chunk: 2
DEBUG:src.langgraph_agent:Streamed chunk:

Chunk:  support
Chunk:  service
Chunk:  operates
Chunk:  
Chunk: 2
Chunk: 4
Chunk: /
Chunk: 7
Chunk: .
Chunk: 


' The phone number for DataNinja Support is 1-800-DATA-NINJA, and this support service operates 24/7.'

In [3]:
graph = agent_graph.get_graph().draw_mermaid()
# Display the graph in Jupyter

import base64
from IPython.display import Image, display

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    return Image(url="https://mermaid.ink/img/" + base64_string)

img = mm(graph)

display(img)

In [ ]:
initial_state = {
        "messages": [HumanMessage(content=user_prompt)],
    }

agent_graph = create_agent_graph(tools, assistant_system_prompt, False)
# Test the agent graph with a user prompt
response = run_agent_graph(agent_graph, initial_state)
print('\n\n\n')
print(response)

TypeError: run_agent_graph() takes 2 positional arguments but 3 were given